<a href="https://colab.research.google.com/github/ArjavJ123/asteroid_alert/blob/main/asteroid_alert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data.dataset import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
data = pd.read_csv("/content/gdrive/MyDrive/asteroid_alert/nasa.csv")

In [ ]:
data.drop(data.columns[[0,1,5,6,7,8,9,10,11,12,14,15,17,18,19,20,21,22,38]],axis=1,inplace=True)
data.shape

data['Hazardous'] = data['Hazardous'].map({True: 1, False: 0})
data_t=torch.tensor(data.values, dtype=torch.float32)
data.head()


,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Relative Velocity km per sec,Miss Dist.(Astronomical),Orbit Uncertainity,Minimum Orbit Intersection,Jupiter Tisserand Invariant,Epoch Osculation,Eccentricity,Semi Major Axis,Inclination,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion,Hazardous
0,21.6,0.127220,0.284472,6.115834,0.419483,5,0.025282,4.634,2458000.5,0.425549,1.407011,6.025981,314.373913,609.599786,0.808259,57.257470,2.005764,2.458162e+06,264.837533,0.590551,1
1,21.3,0.146068,0.326618,18.113985,0.383014,3,0.186935,5.457,2458000.5,0.351674,1.107776,28.412996,136.717242,425.869294,0.718200,313.091975,1.497352,2.457795e+06,173.741112,0.845330,0
2,20.3,0.231502,0.517654,7.590711,0.050956,0,0.043058,4.557,2458000.5,0.348248,1.458824,4.237961,259.475979,643.580228,0.950791,248.415038,1.966857,2.458120e+06,292.893654,0.559371,1
3,27.4,0.008801,0.019681,11.173874,0.285322,6,0.005512,5.093,2458000.5,0.216578,1.255903,7.905894,57.173266,514.082140,0.983902,18.707701,1.527904,2.457902e+06,68.741007,0.700277,0
4,21.6,0.127220,0.284472,9.840831,0.407832,1,0.034798,5.154,2458000.5,0.210448,1.225615,16.793382,84.629307,495.597821,0.967687,158.263596,1.483543,2.457814e+06,135.142133,0.726395,1


In [ ]:
x_train=torch.ones(3749,39)
y_train=torch.ones(3749)
x_val=torch.ones(938,39)
y_val=torch.ones(938)
x_train=data_t[1 : 3749 , : -1]
y_train=data_t[1 : 3749 , -1]
x_val=data_t[3750 : , : -1]
y_val=data_t[3750 : , -1]
x_train= (x_train-torch.mean(x_train))/torch.std(x_train)
x_val=(x_val-torch.mean(x_val))/torch.std(x_val)

print(x_train[0:5])
print(x_val.shape)


tensor([[-0.3334, -0.3334, -0.3334, -0.3334, -0.3334, -0.3334, -0.3334, -0.3334,
          3.0004, -0.3334, -0.3334, -0.3334, -0.3332, -0.3328, -0.3334, -0.3330,
         -0.3334,  3.0001, -0.3332, -0.3334],
        [-0.3334, -0.3334, -0.3334, -0.3334, -0.3334, -0.3334, -0.3334, -0.3334,
          3.0004, -0.3334, -0.3334, -0.3334, -0.3331, -0.3326, -0.3334, -0.3331,
         -0.3334,  3.0005, -0.3330, -0.3334],
        [-0.3334, -0.3334, -0.3334, -0.3334, -0.3334, -0.3334, -0.3334, -0.3334,
          3.0004, -0.3334, -0.3334, -0.3334, -0.3333, -0.3327, -0.3334, -0.3334,
         -0.3334,  3.0002, -0.3333, -0.3334],
        [-0.3334, -0.3334, -0.3334, -0.3334, -0.3334, -0.3334, -0.3334, -0.3334,
          3.0004, -0.3334, -0.3334, -0.3334, -0.3333, -0.3328, -0.3334, -0.3332,
         -0.3334,  3.0001, -0.3332, -0.3334],
        [-0.3334, -0.3334, -0.3334, -0.3334, -0.3334, -0.3334, -0.3334, -0.3334,
          3.0004, -0.3334, -0.3334, -0.3334, -0.3332, -0.3327, -0.3334, -0.3332,
      

In [ ]:
train_data=TensorDataset(x_train,y_train)
val_data=TensorDataset(x_val,y_val)
batchsize=64
train_dataLoader=DataLoader(dataset=train_data,batch_size=batchsize)
val_dataLoader=DataLoader(dataset=val_data,batch_size=batchsize)

In [ ]:
seq_model=nn.Sequential(
    nn.Linear(20,5),
    nn.Tanh(),
    nn.Linear(5,1),
    nn.Sigmoid()
)
seq_model

Sequential(
  (0): Linear(in_features=20, out_features=5, bias=True)
  (1): Tanh()
  (2): Linear(in_features=5, out_features=1, bias=True)
  (3): Sigmoid()
)

In [ ]:
optimizer=optim.SGD(seq_model.parameters(), lr = 1e-2)


In [ ]:
def training_loop(n_epoch,model,loss_fn,optimizer,train_data) :
  for epoch in range(1,n_epoch+1):
      x_train,y_train=td
      yt_pred=model(x_train)
      yt_pred = yt_pred.reshape(y_train.shape)
      optimizer.zero_grad()
      loss=loss_fn(yt_pred,y_train)
      loss.backward()
      optimizer.step()
      if epoch==1 or epoch%1000==0 :
        print(f"Epoch {epoch}, loss {loss.item():.4f}")
  for i in range(len(yt_pred)) :
    if yt_pred[i]>=0.5 :
      yt_pred[i]=1
    elif yt_pred[i]<0.5 :
      yt_pred[i]=0
  return yt_pred

In [ ]:
def accuracy(pred,given) :
  count=0
  for i in range(len(pred)) :
    if pred[i]==given[i] :
      count+=1
  perac= count/len(pred)*100
  print("accuracy : ", perac)

In [ ]:
count = 0
for td in train_dataLoader :
  print(count)
  x_train,y_train=td
  y_pred = training_loop(
      n_epoch=5000,
      model=seq_model,
      loss_fn= nn.BCELoss(),
      optimizer=optimizer,
      train_data=td
  )
  count+=1 
  accuracy(y_pred,y_train)

0
Epoch 1, loss 0.7122
Epoch 1000, loss 0.5253
Epoch 2000, loss 0.5253
Epoch 3000, loss 0.5253
Epoch 4000, loss 0.5253
Epoch 5000, loss 0.5253
accuracy :  78.125
1
Epoch 1, loss 0.5651
Epoch 1000, loss 0.5623
Epoch 2000, loss 0.5623
Epoch 3000, loss 0.5623
Epoch 4000, loss 0.5623
Epoch 5000, loss 0.5623
accuracy :  75.0
2
Epoch 1, loss 0.5280
Epoch 1000, loss 0.5253
Epoch 2000, loss 0.5253
Epoch 3000, loss 0.5253
Epoch 4000, loss 0.5253
Epoch 5000, loss 0.5253
accuracy :  78.125
3
Epoch 1, loss 0.4657
Epoch 1000, loss 0.4588
Epoch 2000, loss 0.4588
Epoch 3000, loss 0.4588
Epoch 4000, loss 0.4588
Epoch 5000, loss 0.4588
accuracy :  82.8125
4
Epoch 1, loss 0.5817
Epoch 1000, loss 0.5623
Epoch 2000, loss 0.5623
Epoch 3000, loss 0.5623
Epoch 4000, loss 0.5623
Epoch 5000, loss 0.5623
accuracy :  75.0
5
Epoch 1, loss 0.5108
Epoch 1000, loss 0.5047
Epoch 2000, loss 0.5047
Epoch 3000, loss 0.5047
Epoch 4000, loss 0.5047
Epoch 5000, loss 0.5047
accuracy :  79.6875
6
Epoch 1, loss 0.4620
Epoch 1

In [ ]:
count = 0
for vd in val_dataLoader :
  print(count)
  x_val,y_val=vd
  yv_pred= seq_model(x_val)
  yv_pred=yv_pred.reshape(y_val.shape)
  for i in range(len(yv_pred)) :
    if yv_pred[i] >= 0.5 :
      yv_pred[i] = 1
    elif yv_pred[i] <0.5 :
      yv_pred[i] = 0
  accuracy(yv_pred,y_val)
  count+=1

0
accuracy :  87.5
1
accuracy :  81.25
2
accuracy :  87.5
3
accuracy :  89.0625
4
accuracy :  87.5
5
accuracy :  84.375
6
accuracy :  87.5
7
accuracy :  85.9375
8
accuracy :  89.0625
9
accuracy :  89.0625
10
accuracy :  84.375
11
accuracy :  92.1875
12
accuracy :  81.25
13
accuracy :  89.0625
14
accuracy :  87.8048780487805
